In [1]:
import pandas as pd
import os

# os.environ['PYSPARK_SUBMIT_ARGS'] = "--master mymaster --total-executor 2 --conf 'spark.driver.extraJavaOptions=-Dhttp.proxyHost=proxy.mycorp.com-Dhttp.proxyPort=1234' -Dhttp.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 -Dhttps.proxyHost=proxy.mycorp.com -Dhttps.proxyPort=1234 -Dhttps.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 pyspark-shell"
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, isnan, when, count
from pyspark.ml.feature import Imputer, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import os

from DataLoader import DataLoader
from DataSetEnum import DataSet


In [7]:
loader = DataLoader(data_set=DataSet.VALIDATION)
train_df = loader.df

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=30067Kb max_used=30091Kb free=101004Kb
 bounds [0x000000010a1e0000, 0x000000010bf70000, 0x00000001121e0000]
 total_blobs=11082 nmethods=10130 adapters=863
 compilation: disabled (not enough contiguous free space left)


24/03/11 15:00:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/train-6.csv
24/03/11 15:00:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/

Removed 0 duplicate rows.


In [3]:
# Count missing values in each column
missing_value_counts = train_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train_df.columns])

# Display the count of missing values for each column
missing_value_counts.show()

24/03/11 14:56:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/validation_hidden.csv


+---+----------+--------------+-------------+----+-----------------+---------------+-----------+-----------+--------------+-------------------+
| id|product_id|product_parent|product_title|vine|verified_purchase|review_headline|review_body|review_date|marketplace_id|product_category_id|
+---+----------+--------------+-------------+----+-----------------+---------------+-----------+-----------+--------------+-------------------+
|  0|         0|             0|           10|   0|                0|            108|          0|          0|           146|                133|
+---+----------+--------------+-------------+----+-----------------+---------------+-----------+-----------+--------------+-------------------+



In [4]:
train_df.groupBy("vine").count().show()

+----+-----+
|vine|count|
+----+-----+
|   Y|    3|
|   N| 1245|
+----+-----+



In [5]:
train_df.groupBy("verified_purchase").count().show()

+-----------------+-----+
|verified_purchase|count|
+-----------------+-----+
|                Y|  798|
|                N|  450|
+-----------------+-----+



In [6]:
train_df.groupBy("label").count().show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `label` cannot be resolved. Did you mean one of the following? [`id`, `vine`, `review_date`, `product_id`, `review_body`].;
'Aggregate ['label], ['label, count(1) AS count#282L]
+- Filter isnotnull(verified_purchase#78)
   +- Filter isnotnull(vine#66)
      +- Project [id#0, product_id#1, product_parent#2, product_title#3, vine#66, CASE WHEN verified_purchase#5 IN (Y,N) THEN verified_purchase#5 ELSE cast(null as string) END AS verified_purchase#78, review_headline#6, review_body#7, review_date#8, marketplace_id#9, product_category_id#10]
         +- Project [id#0, product_id#1, product_parent#2, product_title#3, CASE WHEN vine#4 IN (Y,N) THEN vine#4 ELSE cast(null as string) END AS vine#66, verified_purchase#5, review_headline#6, review_body#7, review_date#8, marketplace_id#9, product_category_id#10]
            +- Relation [id#0,product_id#1,product_parent#2,product_title#3,vine#4,verified_